# 1차 해결
- 1차 리뷰, 이름, 점수 가져오기
- 2차 다른 요소들()
- 3차 페이지 왔다갔다 구현

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import time
                        
# ChromeDriver 경로 설정
chrome_driver_path = 'C:/chromedriver-win64/chromedriver.exe'  # 실제 chromedriver 경로로 변경
chrome_service = Service(chrome_driver_path)

# Selenium 옵션 설정
options = Options()
# options.add_argument("--headless")  # 브라우저를 띄우지 않음 (주석 처리 또는 삭제)
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Selenium WebDriver 초기화
driver = webdriver.Chrome(service=chrome_service, options=options)

# 브라우저 창 크기 설정
driver.set_window_size(1920, 1080)  # 예: 1920x1080 해상도

# 타겟 URL
url = "https://www.airbnb.com/rooms/28159508?category_tag=Tag%3A8678&check_in=2024-05-13&check_out=2024-05-15&enable_m3_private_room=true&federated_search_id=f6713e62-d621-4202-8a81-4798dea79626&locale=en&photo_id=1678755529&previous_page_section_name=1000&search_mode=regular_search&source_impression_id=p3_1715521425_BHa2l4aN25ZU%2FUOl"
driver.get(url)

# 페이지 로딩을 위해 잠시 대기
time.sleep(1)

# 번역 설정 모달 창 닫기
try:
    translation_modal_close_button = WebDriverWait(driver, 1).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="Close"]'))
    )
    translation_modal_close_button.click()
    time.sleep(1)
except Exception as e:
    print("Translation modal did not appear or could not be closed:", e)

# 페이지 소스를 BeautifulSoup으로 파싱
soup = BeautifulSoup(driver.page_source, 'html.parser')

# 방 이름 크롤링
room_name_tag = soup.find('h2', class_='hpipapi')
room_name = room_name_tag.get_text() if room_name_tag else "Room name not found"

# 점수 크롤링
score_tag = soup.find('div', {'aria-hidden': 'true'}, string="4.95")
score = score_tag.get_text() if score_tag else "Score not found"

# 리뷰 버튼 클릭
try:
    review_button = WebDriverWait(driver, 1).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.r1c705dw.atm_c8_1uc0753.atm_g3_lonqig.atm_fr_r7vles.atm_rd_8stvzk.atm_vv_qvpr2i[aria-hidden="true"]'))
    )
    # ActionChains를 사용하여 클릭 시도
    actions = ActionChains(driver)
    actions.move_to_element(review_button).click().perform()
except Exception as e:
    print("Failed to find or click the review button:", e)
    driver.quit()
    exit()

# 모달 창이 열릴 때까지 대기
try:
    scroll_panel = WebDriverWait(driver, 1).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-testid="pdp-reviews-modal-scrollable-panel"]'))
    )
except Exception as e:
    print("Failed to load the reviews modal:", e)
    driver.quit()
    exit()

# 처음에 강제로 스크롤 내리기
driver.execute_script("""
    arguments[0].scrollTop = arguments[0].scrollHeight;
    arguments[0].dispatchEvent(new Event('scroll'));
""", scroll_panel)
time.sleep(1)  # 스크롤 후 로딩을 위해 잠시 대기

# 무한 스크롤을 통해 모든 리뷰를 로드
last_height = driver.execute_script("return arguments[0].scrollHeight", scroll_panel)


while True:
    # 스크롤을 내리는 동작 (패널의 마지막 부분으로 이동)
    driver.execute_script("arguments[0].scrollIntoView(false);", scroll_panel)
    time.sleep(1)  # 스크롤 후 로딩을 위해 잠시 대기

    # 새로운 스크롤 높이를 가져오기
    new_height = driver.execute_script("return arguments[0].scrollHeight", scroll_panel)
    
    if new_height == last_height:
        break
    last_height = new_height
# 페이지 소스를 다시 BeautifulSoup으로 파싱
soup = BeautifulSoup(driver.page_source, 'html.parser')

# 리뷰 크롤링
review_container = soup.find('div', {'data-testid': 'pdp-reviews-modal-scrollable-panel'})
if review_container:
    review_tags = review_container.find_all('span', class_='lrl13de atm_kd_19r6f69_24z95b atm_kd_19r6f69_1xbvphn_1oszvuo dir dir-ltr')
    reviews = [review.get_text().strip() for review in review_tags]
else:
    reviews = []

#driver.quit()

print(f"Room Name: {room_name}")
print(f"Score: {score}")
print("Reviews:")
for review in reviews:
    print(review)

print(len(reviews))


Translation modal did not appear or could not be closed: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.202)
Stacktrace:
	GetHandleVerifier [0x00007FF786411562+60802]
	(No symbol) [0x00007FF78638AC62]
	(No symbol) [0x00007FF786247CE4]
	(No symbol) [0x00007FF78621DFDF]
	(No symbol) [0x00007FF7862C1E57]
	(No symbol) [0x00007FF7862D98D1]
	(No symbol) [0x00007FF7862BA923]
	(No symbol) [0x00007FF786288FEC]
	(No symbol) [0x00007FF786289C21]
	GetHandleVerifier [0x00007FF7867141FD+3217949]
	GetHandleVerifier [0x00007FF786756197+3488183]
	GetHandleVerifier [0x00007FF78674F11F+3459391]
	GetHandleVerifier [0x00007FF7864CB926+823622]
	(No symbol) [0x00007FF786395FFF]
	(No symbol) [0x00007FF786390F24]
	(No symbol) [0x00007FF7863910B2]
	(No symbol) [0x00007FF786381904]
	BaseThreadInitThunk [0x00007FFDB2B47344+20]
	RtlUserThreadStart [0x00007FFDB46A26B1+33]



NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.202)
Stacktrace:
	GetHandleVerifier [0x00007FF786411562+60802]
	(No symbol) [0x00007FF78638AC62]
	(No symbol) [0x00007FF786247CE4]
	(No symbol) [0x00007FF78621DFDF]
	(No symbol) [0x00007FF7862C1E57]
	(No symbol) [0x00007FF7862D98D1]
	(No symbol) [0x00007FF7862BA923]
	(No symbol) [0x00007FF786288FEC]
	(No symbol) [0x00007FF786289C21]
	GetHandleVerifier [0x00007FF7867141FD+3217949]
	GetHandleVerifier [0x00007FF786756197+3488183]
	GetHandleVerifier [0x00007FF78674F11F+3459391]
	GetHandleVerifier [0x00007FF7864CB926+823622]
	(No symbol) [0x00007FF786395FFF]
	(No symbol) [0x00007FF786390F24]
	(No symbol) [0x00007FF7863910B2]
	(No symbol) [0x00007FF786381904]
	BaseThreadInitThunk [0x00007FFDB2B47344+20]
	RtlUserThreadStart [0x00007FFDB46A26B1+33]


# 임시 2차
+ 다른 요소들 받으려고 하는중

In [25]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import time

################################################
########### chrome_driver_path 바꿔야함
# ChromeDriver 경로 설정
chrome_driver_path = 'C:/chromedriver-win64/chromedriver.exe'  # 실제 chromedriver 경로로 변경
chrome_service = Service(chrome_driver_path)

# Selenium 옵션 설정
options = Options()
# options.add_argument("--headless")  # 브라우저를 띄우지 않음 (주석 처리 또는 삭제)
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Selenium WebDriver 초기화
driver = webdriver.Chrome(service=chrome_service, options=options)

# 브라우저 창 크기 설정
driver.set_window_size(1920, 1080)  # 예: 1920x1080 해상도

# 타겟 URL
url = "https://www.airbnb.com/rooms/28159508?category_tag=Tag%3A8678&check_in=2024-05-13&check_out=2024-05-15&enable_m3_private_room=true&federated_search_id=f6713e62-d621-4202-8a81-4798dea79626&locale=en&photo_id=1678755529&previous_page_section_name=1000&search_mode=regular_search&source_impression_id=p3_1715521425_BHa2l4aN25ZU%2FUOl"
driver.get(url)

# 페이지 로딩을 위해 잠시 대기
time.sleep(1)

# 번역 설정 모달 창 닫기
try:
    translation_modal_close_button = WebDriverWait(driver, 1).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="Close"]'))
    )
    translation_modal_close_button.click()
    time.sleep(1)
except Exception as e:
    print("Translation modal did not appear or could not be closed:", e)

# 페이지 소스를 BeautifulSoup으로 파싱
soup = BeautifulSoup(driver.page_source, 'html.parser')

# 방 이름 크롤링
room_name_tag = soup.find('h2', class_='hpipapi')
room_name = room_name_tag.get_text() if room_name_tag else "Room name not found"

# 점수 크롤링
score_tag = soup.find('div', {'aria-hidden': 'true'}, string="4.95")
score = score_tag.get_text() if score_tag else "Score not found"

# 리뷰 버튼 클릭
try:
    review_button = WebDriverWait(driver, 1).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.r1c705dw.atm_c8_1uc0753.atm_g3_lonqig.atm_fr_r7vles.atm_rd_8stvzk.atm_vv_qvpr2i[aria-hidden="true"]'))
    )
    # ActionChains를 사용하여 클릭 시도
    actions = ActionChains(driver)
    actions.move_to_element(review_button).click().perform()
except Exception as e:
    print("Failed to find or click the review button:", e)
    driver.quit()
    exit()

# 모달 창이 열릴 때까지 대기
try:
    scroll_panel = WebDriverWait(driver, 1).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-testid="pdp-reviews-modal-scrollable-panel"]'))
    )
except Exception as e:
    print("Failed to load the reviews modal:", e)
    driver.quit()
    exit()

# 처음에 강제로 스크롤 내리기
driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_panel)
time.sleep(1)  # 스크롤 후 로딩을 위해 잠시 대기

# 무한 스크롤을 통해 모든 리뷰를 로드
last_height = driver.execute_script("return arguments[0].scrollHeight", scroll_panel)

while True:
    # 스크롤을 내리는 동작 (패널의 마지막 부분으로 이동)
    driver.execute_script("arguments[0].scrollIntoView(false);", scroll_panel)
    time.sleep(1)  # 스크롤 후 로딩을 위해 잠시 대기

    # 새로운 스크롤 높이를 가져오기
    new_height = driver.execute_script("return arguments[0].scrollHeight", scroll_panel)
    
    if new_height == last_height:
        break
    last_height = new_height

# 페이지 소스를 다시 BeautifulSoup으로 파싱
soup = BeautifulSoup(driver.page_source, 'html.parser')

# 각 항목의 점수 크롤링
ratings = {}
for rating in ['Cleanliness', 'Accuracy', 'Check-in', 'Communication', 'Location', 'Value']:
    rating_tag = soup.find('div', string=rating)
    if rating_tag:
        score_tag = rating_tag.find_next('span')
        if score_tag:
            ratings[rating] = score_tag.get_text()

# 리뷰 크롤링
review_container = soup.find('div', {'data-testid': 'pdp-reviews-modal-scrollable-panel'})
if review_container:
    review_tags = review_container.find_all('div', class_='r1are2x1')
    reviews = []
    for review_tag in review_tags:
        review_text_tag = review_tag.find('span', {'class': 'll4r2nl dir dir-ltr'})
        if review_text_tag:
            review_text = review_text_tag.get_text().strip()
        else:
            review_text = "No review text found"
        stars = len(review_tag.find_all('span', {'aria-label': 'Star'}))
        reviews.append({'text': review_text, 'stars': stars})
else:
    reviews = []

driver.quit()

print(f"Room Name: {room_name}")
print(f"Score: {score}")
print("Ratings:")
for key, value in ratings.items():
    print(f"{key}: {value}")

print("Reviews:")
for review in reviews:
    print(f"Stars: {review['stars']}, Review: {review['text']}")

print(len(reviews))


Room Name: Room in Elizabeth, New Jersey, United States
Score: 4.95
Ratings:
Cleanliness: Rated 5.0 out of 5 stars for accuracy
Accuracy: Rated 5.0 out of 5 stars for check-in
Check-in: 1 guest
Communication: Rated 4.8 out of 5 stars for location
Location: 
Value: Rating, 5 stars
Reviews:
Stars: 0, Review: No review text found
Stars: 0, Review: No review text found
Stars: 0, Review: No review text found
Stars: 0, Review: No review text found
Stars: 0, Review: No review text found
Stars: 0, Review: No review text found
Stars: 0, Review: No review text found
Stars: 0, Review: No review text found
Stars: 0, Review: No review text found
Stars: 0, Review: No review text found
Stars: 0, Review: No review text found
Stars: 0, Review: No review text found
Stars: 0, Review: No review text found
Stars: 0, Review: No review text found
Stars: 0, Review: No review text found
Stars: 0, Review: No review text found
Stars: 0, Review: No review text found
Stars: 0, Review: No review text found
Stars: 0

# 3차 초반에 구현이라 아직 구현 못함

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
from bs4 import BeautifulSoup

# Chrome Driver 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 백그라운드에서 실행
chrome_options.add_argument("--start-maximized")  # 최대화된 상태로 시작
chrome_options.add_experimental_option("detach", True)  # 스크립트 종료 후에도 브라우저 유지
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])  # 불필요한 로그 제외
chrome_service = Service('C:/chromedriver-win64/chromedriver.exe')  # 크롬 드라이버 경로

def get_listing_details(url):
    driver = webdriver.Chrome(service=chrome_service, options=chrome_options)
    driver.get(url)
    
    # 페이지 로드 대기
    wait = WebDriverWait(driver, 1)  
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-testid="property-card"]')))
    except TimeoutException:
        print("페이지 로딩 시간 초과")
        driver.quit()
        return []

    listings = []
    
    while True:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        listing_elements = soup.find_all('div', {'data-testid': 'property-card'})
        print(f'Found {len(listing_elements)} listing elements.')

        for listing in listing_elements:
            try:
                # 숙소 상세 페이지로 이동
                listing_link = listing.find('a', {'data-testid': 'title-link'})['href']
                driver.get(f"https://www.airbnb.co.kr{listing_link}")
                time.sleep(1)  # 페이지 로딩 시간 대기

                # 숙소 상세 정보 추출
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                room_name = soup.find('h1', {'data-testid': 'title'}).text.strip()
                review_score = soup.find('span', {'aria-label': lambda text: text and "별점" in text}).text.strip()
                reviews = []
                
                # 모든 리뷰 페이지 탐색
                while True:
                    review_elements = soup.find_all('div', {'data-review-id': True})

                    for element in review_elements:
                        review_text = element.find('span', {'class': '_1epc9dn0'}).text
                        reviews.append(review_text)

                    # 다음 리뷰 페이지로 이동
                    try:
                        next_button = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="다음 페이지"]')
                        next_button.click()
                        time.sleep(5)  # 페이지 로딩 시간 대기
                        soup = BeautifulSoup(driver.page_source, 'html.parser')
                    except:
                        break  # 더 이상 다음 페이지가 없으면 종료

                listings.append({
                    'room_name': room_name,
                    'review_score': review_score,
                    'reviews': reviews
                })

                # 뒤로 가기
                driver.back()
                time.sleep(5)  # 페이지 로딩 시간 대기
                soup = BeautifulSoup(driver.page_source, 'html.parser')
            except Exception as e:
                print(f"Error: {e}")
                continue

        # 다음 페이지로 이동
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, 'a[aria-label="다음 페이지"]')
            next_button.click()
            time.sleep(1)  # 페이지 로딩 시간 대기
        except:
            break  # 더 이상 다음 페이지가 없으면 종료

    driver.quit()
    return listings

def save_listings_to_csv(url, output_file):
    listings = get_listing_details(url)
    if listings:
        df = pd.DataFrame(listings)
        df.to_csv(output_file, index=False)
        print(f"Saved listings to {output_file}")
    else:
        print("No listings found.")

# LA 숙소 목록 URL
url = 'https://www.airbnb.co.kr/s/%EB%89%B4%EC%9A%95/homes?place_id=ChIJOwg_06VPwokRYv534QaPC8g&refinement_paths%5B%5D=%2Fhomes&checkin=2024-05-13&checkout=2024-05-15&date_picker_type=calendar&adults=0&children=0&infants=0&pets=0&search_type=AUTOSUGGEST'
save_listings_to_csv(url, 'airbnb_la_listings.csv')
